In [1]:
#projeto tópicos teste 1

In [2]:
import gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback

# Função para criar o ambiente do jogo, agora com pré-processamento
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    # Adiciona os wrappers para otimizar a imagem
    env = ResizeObservation(env, shape=84)
    env = GrayScaleObservation(env, keep_dim=True)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    return env

# Cria 4 ambientes de jogo para treinar em paralelo
vec_env = make_vec_env(make_env, n_envs=4, vec_env_cls=DummyVecEnv)

# Agrupa 4 frames de tela para dar ao agente a noção de movimento
env = VecFrameStack(vec_env, n_stack=4)

# Configura o salvamento automático do modelo a cada 500.000 passos
checkpoint_callback = CheckpointCallback(save_freq=50000, save_path='./train/', name_prefix='mario_model')

# Instancia o modelo PPO
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log="./mario_tensorboard/")

# Inicia o treinamento (reduzimos para 1 milhão para um primeiro teste)
model.learn(total_timesteps=200000, callback=checkpoint_callback)

Using cpu device
Wrapping the env in a VecTransposeImage.
Logging to ./mario_tensorboard/PPO_3
-----------------------------
| time/              |      |
|    fps             | 305  |
|    iterations      | 1    |
|    time_elapsed    | 26   |
|    total_timesteps | 8192 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 134         |
|    iterations           | 2           |
|    time_elapsed         | 121         |
|    total_timesteps      | 16384       |
| train/                  |             |
|    approx_kl            | 0.020353284 |
|    clip_fraction        | 0.241       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.93       |
|    explained_variance   | -0.00123    |
|    learning_rate        | 0.0003      |
|    loss                 | 14.8        |
|    n_updates            | 10          |
|    policy_gradient_loss | 0.0106      |
|    value_loss        

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.61e+04    |
|    ep_rew_mean          | 789         |
| time/                   |             |
|    fps                  | 93          |
|    iterations           | 12          |
|    time_elapsed         | 1047        |
|    total_timesteps      | 98304       |
| train/                  |             |
|    approx_kl            | 0.020495228 |
|    clip_fraction        | 0.192       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.71       |
|    explained_variance   | 0.916       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.01        |
|    n_updates            | 110         |
|    policy_gradient_loss | 0.00693     |
|    value_loss           | 6.14        |
-----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 1.41e+04

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.45e+04    |
|    ep_rew_mean          | 887         |
| time/                   |             |
|    fps                  | 91          |
|    iterations           | 22          |
|    time_elapsed         | 1961        |
|    total_timesteps      | 180224      |
| train/                  |             |
|    approx_kl            | 0.020069472 |
|    clip_fraction        | 0.143       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.8        |
|    explained_variance   | 0.933       |
|    learning_rate        | 0.0003      |
|    loss                 | 1.32        |
|    n_updates            | 210         |
|    policy_gradient_loss | 0.00906     |
|    value_loss           | 3.58        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.45e+

In [9]:
import gym
import gym_super_mario_bros
from nes_py.wrappers import JoypadSpace
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from gym.wrappers import GrayScaleObservation, ResizeObservation
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack # Linha corrigida

# A função para criar o ambiente deve ser IDÊNTICA à do treinamento
def make_env():
    env = gym_super_mario_bros.make('SuperMarioBros-v0')
    env = ResizeObservation(env, shape=84)
    env = GrayScaleObservation(env, keep_dim=True)
    env = JoypadSpace(env, SIMPLE_MOVEMENT)
    return env

# Crie o ambiente base
env = make_env()

# Envolva o ambiente nos mesmos wrappers de vetorização e empilhamento de frames do treino
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, n_stack=4)


# --- CARREGUE SEU MELHOR MODELO ---
# Altere o nome do arquivo para o checkpoint que você quer testar
model = PPO.load('./train/mario_model_200000_steps.zip')

# Inicia a simulação para assistir
obs = env.reset()
done = False
while not done:
    # O modelo treinado prevê a melhor ação
    action, _states = model.predict(obs, deterministic=True)
    
    # A ação é executada no jogo
    obs, rewards, done, info = env.step(action)
    
    # A tela do jogo é exibida para você
    env.render()

# Fecha a janela do jogo
env.close()